# Host Based Intrusion Detection System- Sequential Data

In [ ]:
#Import Dependencies

import tensorflow as tf
from numpy import mean,std
from tensorflow.python.client import device_lib
import time
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, TimeDistributed,Flatten,BatchNormalization,MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU,LSTM
from keras import regularizers
from keras.regularizers import l2

from keras.layers import Dropout
import numpy as np
import random
import os,glob
import numpy,math
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import merge
from sklearn import metrics


from keras.layers import Activation, Dense
from sklearn.metrics import confusion_matrix

In [ ]:
## abstract the normal training data
start_time=time.time()

root=('C://Users//a.chawla//Desktop//IDS//ADFA-LD//Training_Data_Master')
os.chdir(root)
data =[]

#print(os.listdir(root))
for fname in glob.glob("*"):
    file = open(fname,"r")                     ## open and read the file 
    callSeqString = (file.read())
    callSeq = callSeqString.split()            ## Split the characters 
    #print(callSeq)
    callSeqInt = list(map(int, callSeq))       ## Convert Strings to Integers
    #print (callSeqInt)
    data.append([callSeqInt])
    
x1=[]
for i in range(0,len(data)) :
        x1.append(data[i][0])


print(time.time()-start_time)

In [ ]:
'''Create the Multilayer Perceptron model'''

nWords = 340
model = Sequential()

model.add(Embedding(nWords,32,input_length=None))

model.add(layers.Conv1D(64, 2, padding = 'same', activation="relu"))
model.add(layers.MaxPooling1D(5))
model.add(BatchNormalization())
model.add(layers.Conv1D(64, 2, padding = 'same', activation="relu"))
model.add(BatchNormalization())
model.add(layers.Conv1D(64, 2, padding = 'same', activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(GRU(200,return_sequences=True))
model.add(Dropout(0.7))

model.add(TimeDistributed(Dense(nWords+1, activation='softmax')))


model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizers.Adam(lr=0.0001))
print(model.summary())

In [ ]:
##Dynamically pad the sequence and train the model 

start_time=time.time()

startIndex=0
batchsize=32 ## batch size in which training will happen

endIndex=batchsize
totalBatches= int(len(x1)/batchsize)


def train_epoch(x1,startIndex,endIndex):
    losses = []
    for batch_seq in range(totalBatches):
        batch_seq=x1[startIndex:endIndex]
        max_length=max(len(seq) for seq in batch_seq)
        train_seq=pad_sequences(batch_seq,max_length, dtype=int)
    
        
        temp_x=[]
        temp_y=[]
        for seq in train_seq:
            seq_x = seq[None, :-1]
            seq_y = seq[None, 1:, None]
            temp_x.append(seq_x)
            temp_y.append(seq_y)
           
        
        result_x=np.array(temp_x).reshape(batchsize,max_length-1)
        result_y=np.array(temp_y).reshape(batchsize, max_length-1, 1)
        loss = model.train_on_batch(x=result_x, y=result_y)
        losses.append(loss)
        
        
        startIndex=startIndex+batchsize
        endIndex=endIndex+batchsize
        loss = np.mean(losses)
        model.reset_states()  
    return loss
    
    
n_epochs =100
for epoch in range(n_epochs):
    loss = train_epoch(x1,startIndex,endIndex)
    print(epoch+1,loss)
print(time.time()-start_time) 

In [ ]:
##Fetch Attack Data

root=('C://Users//a.chawla//Desktop//IDS//ADFA-LD//Attack_Data_Master')
os.chdir(root)

attackprobSeq =[]
label=[]

for dirname in glob.glob("*"):
    os.chdir(os.path.join(root,dirname))           ## Dynamically change the directory name 
    for fname in glob.glob("*"):
        file = open(fname,"r")                     ## open and read the file
        callSeqString = file.read()
        callSeq = callSeqString.split()            ## Split the characters 
        callSeqInt = list(map(int, callSeq))      ## Convert Strings to Integers
        attackprobSeq.append([callSeqInt])
        label.append(1)

In [ ]:
##Fetch Validation Data

root=('C://Users//a.chawla//Desktop//IDS//ADFA-LD//Validation_Data_Master')
os.chdir(root)

#print(os.listdir(root))
for fname in glob.glob("*"):
    file = open(fname,"r")                     ## open and read the file
    #print(file)
    callSeqString = file.read()
    callSeq = callSeqString.split()            ## Split the characters 
    callSeqInt = list(map(int, callSeq))       ## Convert Strings to Integers
    evalprobSeq.append([callSeqInt])
    label.append(0)

    
evalprobSeq=[]

for i in range(0,len(evalprobSeq)) :
        evalprobSeq.append(evalprobSeq[i][0])      
        
label = np.array(label)

# Function call to calculate the probabilities

In [ ]:
def evaluateProbability(callSeq):
    pSum=0
    encoded = np.array(callSeq)[None, :]
    
    
    # predict a word in the vocabulary
    yhat = model.predict(encoded)
    temp = yhat[0]

    for i in range(0, len(callSeq)-1):
        next = callSeq[i+1]
        p = -math.log10(temp[i][next])
        pSum = pSum + p
    return pSum/len(callSeq)


In [ ]:
# ## Calculate the probability (lr=0.001)
start_time=time.time()

prob=[]
for x in attackprobSeq:
    p =(evaluateProbability(x))
    prob.append(p)

Mean_Attack= mean(prob[0:745])
Std_dev_Attack= std(prob[0:745])

## print the statistical measures
print(Mean_Attack)
print(Std_dev_Attack)


## Validation Mean and SD
Mean_Val= (mean(prob[746:5118]))
Std_dev_Val= std(prob[746:5118])
print(Mean_Val)
print(Std_dev_Val)
#print(len(prob))

# Calculate the Metrics and plot ROC

In [ ]:
##Calculate FPR and TPR CNN+GRU 200 Units 

Thresh=np.arange(Mean_Val-1,Mean_Attack+1,.02,dtype='float32')

print('No of Threshold values:',len(Thresh))
print(Thresh)

print('Probability values:',prob)
print('---------------')
tpr=[]
fpr=[]
pred=[]

for i in Thresh:
    for value in prob:
        if(value>i):
            pred.append(1)
            
        else:
            pred.append(0)

    
    pred=np.array(pred, dtype=int)
    conf = confusion_matrix(label,pred)

    FP = conf[0,1]
    TN = conf[0,0]
    FN = conf[1,0]
    TP = conf[1,1]
    print('True Negatives:',TN)
    print('False Positives:',FP)
    print('False Negatives:',FN)
    print('True Positives:',TP)
    print('------------')
    
    true_positive_rate=TP/(TP+FN)
    tpr.append(true_positive_rate)
    #print(tpr)
    
    false_positive_rate=FP/(FP+TN)
    fpr.append(false_positive_rate)
    
    pred=[]
         
tpr=np.array(tpr, dtype='float32')
fpr=np.array(fpr,dtype='float32')


print('Detection Rate:',tpr)
print('False Alarm Rate:', fpr)

In [ ]:
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.plot([0.0, 1.0], [0.0, 1.0], color='navy', lw=2, linestyle='--')
roc_auc = metrics.auc(fpr, tpr)

plt.plot(fpr_LSTM,tpr_LSTM,color='deeppink', linestyle=':',lw=2,label='GRU with 200 (AUC = %0.2f)' % roc_auc)
plt.xlabel("False Alarm Rate [%]")
plt.ylabel("Detection Rate [%]")
plt.title('Receiver operating characteristic Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()
